<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br><br><br>

# Listed Volatility and Variance Derivatives

**Wiley Finance (2017)**

Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [@dyjh](http://twitter.com/dyjh) | http://books.tpq.io

<img src="https://hilpisch.com/images/lvvd_cover.png" alt="Listed Volatility and Variance Derivatives" width="30%" align="left" border="0">

# DX Analytics &mdash; Square-Root Diffusion 

## Introduction 

This chapter uses DX Analytics to model the VSTOXX volatility index by a square-root diffusion process as proposed in Gruenbichler and Longstaff (1996) and discussed in chapter _Valuing Volatility Derivatives_. It implements a study over a time period of three months to analyze how well the model performs in replicating market quotes for VSTOXX options.

## Data Import and Selection

The data we are working with is for the first quarter of 2014. The complete data set is contained in the online resources accompanying this book. As usual, some imports first.

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

Next, we read the data from the source into pandas ``DataFrame`` objects.

In [ ]:
h5 = pd.HDFStore('./data/vstoxx_march_2014.h5', 'r')
vstoxx_index = h5['vstoxx_index']  # data for the index itself 
vstoxx_futures = h5['vstoxx_futures']  # data for the futures
vstoxx_options = h5['vstoxx_options']  # data for the options
h5.close()

Inspecting the data sub-set for the VSTOXX index itself, we see that we are dealing with 63 trading days.

In [ ]:
vstoxx_index.info()

In [ ]:
vstoxx_index.tail()

Per trading day, there are eight futures quotes for the eight different maturities of the VSTOXX futures contract. This makes for a total of 504 futures quotes.

In [ ]:
vstoxx_futures['DATE'] = pd.to_datetime(vstoxx_futures['DATE'])

In [ ]:
vstoxx_futures['MATURITY'] = pd.to_datetime(vstoxx_futures['MATURITY'])

In [ ]:
vstoxx_futures.info()

In [ ]:
vstoxx_futures.tail()

By far the biggest data sub-set is the one for the VSTOXX options. There are for each trading day market quotes for puts and calls for eight different maturities and a multitude of different strike prices. This makes for a total of 46960 option quotes for the first quarter of 2014.

In [ ]:
vstoxx_options['DATE'] = pd.to_datetime(vstoxx_options['DATE'])

In [ ]:
vstoxx_options['MATURITY'] = pd.to_datetime(vstoxx_options['MATURITY'])

In [ ]:
vstoxx_options.info()

In [ ]:
vstoxx_options.tail()

Maturity-wise we are dealing with a total of eleven dates. This is due to the fact that at any given time eight maturities for the VSTOXX futures and options contracts are available and we are looking at data for three months.

In [ ]:
third_fridays = sorted(set(vstoxx_futures['MATURITY']))
third_fridays

When it comes to the calibration of the square-root diffusion model for the VSTOXX, it is necessary to work with a selection from the large set of option quotes. The following function implements such a selection procedure, using different conditions to generate a sensible set of option quotes around the forward at-the-money level. The function `srd_get_option_selection()` is used in what follows to select the right sub-set of option quotes for each day during the calibration.

In [ ]:
import sys
sys.path.append('scripts')

In [ ]:
import dx_srd_calibration as dxsrd

In [ ]:
dxsrd.srd_get_option_selection??

## Modeling the VSTOXX Options

The previous chapter illustrates how European options are modeled with DX Analytics based on a geometric Brownian motion model (`dx.geometric_brownian_motion()`). To model the VSTOXX options for the calibration, we just need to replace that model by the square-root diffusion model `dx.square_root_diffusion()`. The respective market environment then needs some additional parameters.

All the code used for the calibration is found in the Python script ``dx_srd_calibration.py`` (see the appendix for the complete script). After some imports, the script starts by defining some general parameters and curves for the market environment. During the calibration process, some of these get updated to reflect the current status of the optimization procedure. 

In [ ]:
!sed -n 11,53p scripts/dx_srd_calibration.py

The function `srd_get_option_models()` creates valuation models for all options in a given selection of option quotes.

In [ ]:
dxsrd.srd_get_option_models??

## Calibration of the VSTOXX Model

Calibration of a parametrized model usually boils down to using global and local optimization algorithms to find parameters that minimize a given target function. This process is discussed in detail in Hilpisch (2015, ch. 11). For the calibration process to follow, we use the helper function `srd_calculate_model_values()` to calculate "at once" the model values for the VSTOXX options at hand. The function parameter `p0` is a tuple since this is what the optimization functions provide as input.

In [ ]:
dxsrd.srd_calculate_model_values??

The target function to be minimized during the calibration is the _mean-squared error_ of the model values given the market quotes of the VSTOXX options. Again, refer to Hilpisch (2015, ch. 11) for details and alternative formulations. The function `srd_mean_squared_error()` implements this concept and uses the function `srd_calculate_model_values()` for the option model value calculations.

In [ ]:
dxsrd.srd_mean_squared_error??

Equipped with the target function to be minimized, we can define the function for the global and local calibration routine itself. The calibration takes place for one or multiple maturities over the pricing date range defined. For example, the function `srd_get_parameter_series()` can calibrate the model (separately) for the two maturities May and June 2014 over the complete first quarter 2014.

In [ ]:
dxsrd.srd_get_parameter_series??

The final step is to start the calibration and collect the calibration results. The calibration we implement is for the 18. April 2014 maturity.

In [ ]:
!sed -n 256,287p scripts/dx_srd_calibration.py

### No Penalization

A visualization of the calibration results tells the whole story. The following figures shows the three square-root diffusion parameters over time and the resulting MSE values. 

In [ ]:
# %run scripts/dx_srd_calibration.py

<img src="./images/dx_srd_cali_1_2014-04-18.png" width="75%">

<p style="font-family: monospace;">Square-root diffusion parameters and MSE values from the calibration to a single maturity (18. April 2014) &mdash; no penalization.

As we can see throughout, the results are quite good given the low MSE values. The mean MSE value is below 0.01 as seen in the following figure.

<img src="images/dx_srd_cali_1_hist_2014-04-18.png" width="75%">
 
<p style="font-family: monospace;">Histogram of the mean-squared errors for the calibration of the square-root diffusion model to a single maturity (18. April 2015) &mdash; no penalization.

### With Penalization

<img src="./images/dx_srd_cali_1_2014-04-18_.png" width="75%">

<p style="font-family: monospace;">Square-root diffusion parameters and MSE values from the calibration to a single maturity (18. April 2014) &mdash; with penalization.

As we can see throughout, the results are quite good given the low MSE values. The mean MSE value is below 0.01 as seen in the following figure.

<img src="images/dx_srd_cali_1_hist_2014-04-18_.png" width="75%">
 
<p style="font-family: monospace;">Histogram of the mean-squared errors for the calibration of the square-root diffusion model to a single maturity (18. April 2015) &mdash; with penalization.

## Conclusions

This chapter uses DX Analytics to model the VSTOXX volatility index by a square-root diffusion process. In similar vein, it is used to model traded European call options on the index to implement a calibration of the VSTOXX model over time. The results show that when calibrating the model to a single options maturity only, the model performs quite well yielding rather low MSE values throughout. The parameter values also seem to be in sensible regions throughout (e.g. `theta` between 15 and 18) and they evolve rather smoothly.

There exist closed form solutions for the price of a European call option in the square-root diffusion model of Gruenbichler and Longstaff (1996) as shown in chapter _Valuing Volatility Derivatives_. For our analysis in this chapter we have nevertheless used the Monte Carlo valuation model of DX Analytics since this approach is more general in that we can easily replace one model by another, maybe more sophisticated, one. This is done in the next chapter where the same study is implemented based on the square-root jump diffusion model presented in chapter _Advanced Modeling of the VSTOXX Index_. The only difference is that a few more parameters need to be taken care of.

## Python Scripts

### `dx_srd_calibration.py`

In [ ]:
!cat scripts/dx_srd_calibration.py

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>